In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import os

#abertura de bases
os.chdir('C:/Users/f8026379/Documents/MANP/1.Alteryx Projects/Capacity Dimensioning/W35_2019')
df_DDS=pd.read_excel('Relatório de Ocupação de Células 4G – Semanal.xlsx',sheet_name='Relatório de Ocupação de Célula')
df_si=pd.read_csv('REPORT_SI_SI_20190926_030002.csv',sep=';',error_bad_lines=False,usecols=['REGIONAL','UF','ANF','MUNICIPIO','IBGE_ID','LOCATION','CS_NAME','LATITUDE','LONGITUDE','AZIMUTH'])
df_si=df_si[df_si['CS_NAME'].notnull()]



C:\Users\f8026379\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [153]:
#criação dos Data Frames reduzidos
df_cells=DataFrame([])
df_cells['Fornecedor']=df_DDS['Fornecedor']
df_cells['Regional']=df_DDS['Regional']
df_cells['UF']=df_DDS['Estado']
df_cells['ANF']=df_DDS['ANF']
df_cells['MUNICIPIO']=df_DDS['Município']
df_cells['IBGE']=df_DDS['IBGE']
df_cells['Site_ID']=df_DDS['BTS/NodeB/ENodeB']
df_cells['Cell']=df_DDS['Célula']
df_cells['Banda']=df_DDS['Banda']
df_cells['Week']=df_DDS['Semana do Ano']
df_cells['Hora']=df_DDS['Hora']
df_cells['Volume']=df_DDS['Vol_Total_Dl_Allop_LT']
df_cells['Cell_Thp']=df_DDS['Throu_Pdcp_Cell_Dl_LT']
df_cells['User_Thp']=df_DDS['Throu_User _Pdcp_Dl_LT']
df_cells['RRCa']=df_DDS['Act_Ue_Data_Dl_LT']
df_cells['RRCc']=df_DDS['User_Endb_Mean_Cell_LT']
df_cells['Utilization']=df_DDS['Util_LT']
df_cells.dropna(axis=0,subset=['Volume'],inplace=True)

df_station=DataFrame([])
df_station['Regional']=df_si['REGIONAL']
df_station['UF']=df_si['UF']
df_station['ANF']=df_si['ANF']
df_station['MUNICIPIO']=df_si['MUNICIPIO']
df_station['IBGE']=df_si['IBGE_ID']
df_station['LOCATION']=df_si['LOCATION']
df_station['Cell']=df_si['CS_NAME']
df_station['LATITUDE']=df_si['LATITUDE']
df_station['LONGITUDE']=df_si['LONGITUDE']
df_station['Azimuth']=df_si['AZIMUTH']
#df_station.drop(axis=0,inplace=True)

#De_para de células para setores
sector={'5':'1','6':'2','7':'3','0':'1','8':'1','9':'1','i':'1','j':'2','k':'3','1':'1','2':'2','3':'3','4':'4','A':'1','B':'2','C':'3','D':'4','E':'1','F':'2','G':'3','H':'4','I':'1','J':'2','K':'3','L':'4','M':'1','N':'2','O':'3','P':'4','Q':'1','R':'2','S':'3','T':'1','U':'2','V':'3','W':'4','X':'1','Y':'2','Z':'3'}

#extrai letra identificadora da célula
def celula(c):
    return c.upper()[-1]

#converte "ND" por frequencia via regra geral
def layer_nd(a,b):
    banda={'1':'2600','2':'2600','3':'2600','4':'2600','A':'2600','B':'2600','C':'2600','D':'2600','E':'700','F':'700','G':'700','H':'700','I':'1800','J':'1800','K':'1800','L':'1800','M':'850','N':'850','O':'850','P':'850','Q':'2100','R':'2100','S':'2100','T':'1800','U':'1800','V':'1800','W':'1800','X':'2600','Y':'2600','Z':'2600'}
    if a=='ND':
        c=banda[b]
    else:
        c=a
    return c

#recupera o nome do site id sem letra da célula
def site_name(s):
    return s[:len(s)-1]

#Recupera nome do site 2G
def Site_2G(x,y):
    return x[-6:]+y

#remove colunas do df que contem a string s
def remove_column(df,s):
    x=df.columns
    c=[]
    for i in x:
        if s in i:
            c.append(i)
        else:
            pass
    return df.drop(columns=c,axis=1)

#criação de colunas auxiliares
df_cells['cell_id'] = df_cells['Cell'].apply(celula)
df_cells['sector']=df_cells['cell_id'].map(sector)
df_cells['Sector_name']=df_cells.Site_ID+ df_cells.sector
df_cells['Layer']=df_cells.apply(lambda row: layer_nd(row['Banda'], row['cell_id']), axis=1)
df_cells['Sector_2G']=df_cells.apply(lambda row: Site_2G(row['Site_ID'], row['sector']), axis=1)


df_station['cell_id'] = df_station['Cell'].apply(celula)
df_station['Site_ID']=df_station['Cell'].apply(site_name)
df_station['sector']=df_station['cell_id'].map(sector)
df_station['Sector_name']=df_station.Site_ID+ df_station.sector
df_station['Sector_2G']=df_station.apply(lambda row: Site_2G(row['Site_ID'], row['sector']), axis=1)

In [205]:
#Join medição e NGNIS
inner_cells=pd.merge(df_station,df_cells,on=['IBGE','Sector_2G'],suffixes=('_l',''),how='inner')
inner_cells.drop_duplicates(subset='Cell',inplace=True)

#Celulas com medição fora do NGNIS
outer_cells=pd.merge(df_station,df_cells,on=['IBGE','Sector_2G'],suffixes=('_l',''),how='right')
outer_cells.drop_duplicates(subset='Cell',inplace=True)
outer_cells=outer_cells[outer_cells.Cell_l.isnull()]
outer_cells.dropna(axis=1,inplace=True)
outer_cells=pd.merge(df_station,outer_cells,on=['Site_ID'],suffixes=('_l',''),how='inner')
outer_cells.Cell_l=outer_cells.Cell
outer_cells.drop_duplicates(subset='Cell',inplace=True)


#União
Base=pd.concat([outer_cells,inner_cells],ignore_index=True,sort=False)
Base=remove_column(Base,'_l')

# Specify a writer
writer = pd.ExcelWriter('Report.xlsx', engine='xlsxwriter')
# Write your DataFrame to a file     
#df_cells.to_excel(writer, 'test',index=False)
#df_station.to_excel(writer, 'Stations',index=False)
Base.to_excel(writer, 'Base',index=False)
inner_cells.to_excel(writer, 'Inner',index=False)
outer_cells.to_excel(writer, 'Outer',index=False)
writer.save()